<a href="https://colab.research.google.com/github/dekovski/GLA_Neural_Lyapunov/blob/main/Example1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Example 1: 2d system**

In [ ]:
!pip install nangs
!pip install torchdiffeq

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import torch
import nangs
import math


device = "cuda" if torch.cuda.is_available() else "cpu"
nangs.__version__, torch.__version__

In [ ]:
# Build your own set of eigenfunctions
x,y = sp.symbols('x y', real=True)
p1 = -x + x**3
p2 = y - y**3
phi1 = (p1 + 2*p2);
phi2 = (2*p1 - 3*p2);
phi = sp.Matrix([phi1,phi2]);
J = phi.jacobian([x,y]);
f = sp.simplify(J.inv()*sp.Matrix([[-0.8,0],[0,-1.2]])*phi)
F = f.jacobian([x,y])

# Linear and nonlinear parts of the dynamics
A = F.subs([(x,0),(y,0)])
fn = f - A*sp.Matrix([x,y])
E,W = np.linalg.eig(np.array(A.T).astype(np.float64))
w1 = W[:,0]; w2 = W[:,1]
l1 = E[0]; l2 = E[1]

# Relevant functions used for computation/comparision
p1= sp.lambdify((x,y),phi1)
p2= sp.lambdify((x,y),phi2)
#g1 = np.dot(w1,fn)[0]; g2 = np.dot(w2,fn)[0]
g1 = np.dot(w1,[x,y]) ; g2 = np.dot(w2,[x,y]);
g1 = sp.lambdify((x,y),g1); g2 = sp.lambdify((x,y),g2)
f = sp.lambdify((x,y),tuple(f))

### 1.1. EDMD for local approximation

In [ ]:
# -*- coding: utf-8 -*-
from sympy.polys.orderings import monomial_key
from sympy.polys.monomials import itermonomials
from numpy.linalg import inv, eig, pinv, det
from scipy.linalg import svd, svdvals, sqrtm
from numpy import diag, dot, real, imag
from IPython import display
import torch

np.random.seed(10)
torch.manual_seed(0)

# Sample data within epsilon nbhd
def sample_data(dic,N,T, eps):
    X = np.empty((T*N,2))
    Y = np.empty((T*N,2))
    thetas = 2*np.pi*np.random.rand(N)
    r = np.random.rand(N)
    P = eps*np.stack([r,r],axis=1)*np.stack([np.sin(thetas),np.cos(thetas)],axis=1)
    Start = P
    for k in range(T):
        X[k*N:(k+1)*N,:] = P
        P = P + dt*np.array(f(P[:,0],P[:,1])).T
        Y[k*N:(k+1)*N,:] = P
    X = basis_fun(X.T)
    Y = basis_fun(Y.T)
    return X,Y, Start

# Dictionary of monomials for EDMD, deg = 10 i.e. 66 total dictionary functions
def basis_fun(X):
    ret = np.ones((len(monomials),X.shape[1]))
    ret[1:,:] = np.array(basis(X[0],X[1])[1:])
    return ret[1:,:]

deg = 3
monomials = sorted(itermonomials([x, y], deg), key=monomial_key('grlex', [y, x]))
basis = sp.lambdify((x,y),monomials,"numpy")

# EDMD
N = 100; T=1000; eps = 0.05; dt=0.01
X,Y,Start = sample_data(basis_fun,N,T,eps=eps)

Fphi = np.eye(X.shape[0])
Xtr = X.copy()
Ytr = Y.copy()

while(Xtr.shape[0]>4):
    A = np.matmul(Ytr,np.linalg.pinv(Xtr))
    mu,phi = eig(A.T)
    res_eig = np.abs(np.matmul(phi.T,Ytr - np.matmul(A,Xtr)))
    res_eig = np.max(res_eig,axis=1)
    #print(np.sort(res_eig))
    s = np.argsort(res_eig)
    phi = phi[:,s]
    m = Xtr.shape[0]
    m = int(m/2)
    phi = phi[:,0:m]
    Fphi = np.matmul(Fphi,phi)
    Xtr = np.matmul(Fphi.T,X)
    Ytr = np.matmul(Fphi.T,Y)

A = np.matmul(Ytr,np.linalg.pinv(Xtr))
mu,phi = eig(A.T)
lam = np.log(mu)/dt
K = np.diag(mu)
phi = np.matmul(Fphi,phi)
res_eig = np.abs(np.matmul(phi.T,Y) - np.matmul(K.T,np.matmul(phi.T,X)))
res_eig = np.max(res_eig,axis=1)
#print(np.sort(res_eig))

# Get the stable eigenmodes
stable = np.argwhere(real(lam)<0).T[0]
phi_stable = phi[:,stable]
lam_stable = lam[stable]
sort_idx = np.argsort(real(lam_stable))
phi_stable = phi_stable[:,sort_idx]
lam_stable = lam_stable[sort_idx]

g = sp.Matrix(real(phi_stable.T)*(abs(real(phi_stable.T))>1e-3))*sp.Matrix(monomials[1:])

In [ ]:
# Use these analytical experssions later for comparision
print(phi1)     # Actual eigenfunction to be learnt
print(-g[-1])   #Local EDMD approximation of phi1 (mind the sign and scaling).

print(lam_stable[-1]) #Eigenvalue approximated by EDMD (should be roughly -0.8)

In [ ]:
#Define the eigenfunction PDE

from nangs import PDE
l1 = real(lam_stable[-1]); # Learnt from EDMD
g1 = sp.lambdify((x,y),g[-1]) # Learnt from EDMD

class Eigen(PDE):
    def computePDELoss(self, inputs, outputs):

        # compute gradients
        grads = self.computeGrads(outputs, inputs)

        # compute loss
        dpdx, dpdy = grads[:, 0], grads[:, 1]
        x, y = inputs[:, 0], inputs[:, 1]
        p = outputs
        u, v = f(x, y)
        return {'pde': 0.01*(-l1*p + u*dpdx + v*dpdy)}

# instantiate pde
pde = Eigen(inputs=('x', 'y'), outputs='p')

### 1.2. Prepare training data using GLA

In [ ]:
# define the sampler

from nangs import RandomSampler

eps=0.45
sampler = RandomSampler({
    'x': [-eps, eps],
    'y': [-eps, eps],
}, device=device, n_samples=1000)

pde.set_sampler(sampler)

In [ ]:
from torchdiffeq import odeint
from nangs import Dirichlet

box = {}
for i in range(2):
  box[pde.inputs[i]]=[-eps, eps]

H = lambda t,X : torch.vstack(f(*tuple(X[0:-1])) + (((1/(t+1e-5))*(-X[-1] + np.exp(-l1*t)*g1(*tuple(X[0:-1])))),))
samples_per_face = 100
T = 100.
X_box = torch.empty(0)
Y_box = torch.empty(0)
for i in range(2):
  #Face A
  dic=box #This is not a deep copy, caution!
  dic[pde.inputs[i]]=[eps,eps]
  X_ = tuple(torch.rand(samples_per_face, )*(lims[1] - lims[0]) + lims[0] for var, lims in dic.items())
  Y_ = odeint(H, torch.vstack(X_ + (torch.zeros(samples_per_face, ),)), torch.tensor([0.,T]))[-1][-1]
  Y_box = torch.hstack((Y_box,Y_))
  X_box = torch.hstack((X_box, torch.vstack(X_)))
  #Face B
  dic=box
  dic[pde.inputs[i]]=[-eps,-eps]
  X_ = tuple(torch.rand(samples_per_face, )*(lims[1] - lims[0]) + lims[0] for var, lims in dic.items())
  Y_ = odeint(H, torch.vstack(X_ + (torch.zeros(samples_per_face),)), torch.tensor([0.,T]))[-1][-1]
  Y_box = torch.hstack((Y_box,Y_))
  X_box = torch.hstack((X_box, torch.vstack(X_)))
  dic[pde.inputs[i]]=[-eps,eps]

#Interior
samples_interior = 1000
X_ = tuple(torch.rand(samples_interior, )*(lims[1] - lims[0]) + lims[0] for var, lims in dic.items())
Y_ = odeint(H, torch.vstack(X_ + (torch.zeros(samples_interior),)), torch.tensor([0.,T]))[-1][-1]
Y_box = torch.hstack((Y_box,Y_))
X_box = torch.hstack((X_box, torch.vstack(X_)))

class Dirichlet(nangs.bocos.boco.Boco):
  def __init__(self, X_box, Y_box, name="dirichlet"):
      super().__init__(name)
      self.X_ = X_box.T
      self.Y_ = Y_box

  def validate(self, inputs, outputs):
      super().validate()

  def computeLoss(self, model, criterion, inputs, outputs):
      y_hat = model(self.X_)[:,0]
      y = self.Y_
      return {self.name: criterion(y, y_hat)}

initial_condition = Dirichlet(X_box.to(device),Y_box.to(device),"faces")
pde.add_boco(initial_condition)

### 1.3. Train

In [ ]:
# solve
from nangs import MLP

LR = 1e-2
N_STEPS = 2000
NUM_LAYERS = 2
NUM_HIDDEN = 128

mlp = MLP(len(pde.inputs), len(pde.outputs), NUM_LAYERS, NUM_HIDDEN).to(device)
optimizer = torch.optim.Adam(mlp.parameters())
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, pct_start=0.1, div_factor=10, final_div_factor=1, total_steps=N_STEPS)

pde.compile(mlp, optimizer, scheduler)
%time hist = pde.solve(N_STEPS)

In [ ]:
# plot loss history
import pandas as pd

df = pd.DataFrame(hist)
fig = plt.figure(dpi=100)
ax = plt.subplot(1,1,1)
ax.set_yscale('log')
df.plot(ax=ax, grid=True)

### 1.4. Evaluate

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

Nd=100
x = np.linspace(-0.45,0.45,Nd)
y = np.linspace(-0.45,0.45,Nd)

_x, _y = np.meshgrid(x, y)

grid = np.stack(np.meshgrid(x, y), -1).reshape(-1, 2)
X = torch.from_numpy(grid).float().to(device)
p = pde.eval(X)
p = p.cpu().numpy().reshape((len(_y),len(_x)))

z_train = Y_box.detach().numpy()
xy_train = X_box.detach().numpy()

### 1.5. Visualize

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

#fig = go.Figure(data=[go.Surface(z=(-w1[0]/2)*(2*_x**3+3*_y**3), x=_x, y=_y, showscale=False, opacity=0.3), go.Surface(z=p, x=_x, y=_y) ])
fig = go.Figure(data=[go.Surface(z=-0.38*(_x**3 - _x - 2*_y**3 + 2*_y), x=_x, y=_y, showscale=True, opacity=0.5,colorbar=dict(lenmode='fraction', len=0.6, thickness=18)), # <-- Actual eigenfunction
                      go.Scatter3d(x = xy_train[0,:], y = xy_train[1,:], z = z_train, mode = 'markers', marker = dict(size = 1.5, color = 'black')), # <-- GLA generated training data
                      #go.Surface(z=p, x=_x, y=_y, opacity=0.3, colorbar=dict(lenmode='fraction', len=0.6, thickness=18)) # <-- DNN eigenfunction
                      ])

fig.update_layout(title='Actual vs learnt eigenfunction',
                  autosize=True,
                  width=600, height=600,
                  margin=dict(l=0, r=0, b=15, t=30))

fig.update_layout(scene = dict(
                    xaxis_title='x1',
                    yaxis_title='x2',
                    zaxis_title='g*(x)'))
fig.update_layout(scene_aspectmode='cube')
fig.update_coloraxes(colorbar_xpad=0)
fig.show()


In [ ]:
# Save if all looks A-OK !
from google.colab import files
torch.save(mlp.state_dict(),'checkpoint_GLA.pth')
files.download('checkpoint_GLA.pth')